In [2]:
import numpy as np
import random
import time
import pandas as pd

# Przygotowanie danych wejściowych

Experiment 1: seed=0, ratio=0.6, range_small=(5, 30), range_large=(80, 110), w=0.5, c1=1.0, c2=1.0 => initial=273, final=268, improve=5

In [3]:
# Pojemność pudełek
BIN_CAPACITY = 180

# Losowa, ale zdeterminowana lista wag: 1000 elementów (60% małych, 40% dużych)
np.random.seed(0)
small_items = np.random.randint(5, 30, size=600)
large_items = np.random.randint(80, 110, size=400)
ITEM_WEIGHTS = np.concatenate([small_items, large_items])
np.random.shuffle(ITEM_WEIGHTS)
ITEM_WEIGHTS = ITEM_WEIGHTS.tolist()

# Liczba przedmiotów
NUM_ITEMS = len(ITEM_WEIGHTS)

print(f"Liczba elementów: {NUM_ITEMS}")
print(f"Pierwsze 10 wag: {ITEM_WEIGHTS[:10]}")


Liczba elementów: 1000
Pierwsze 10 wag: [18, 17, 106, 107, 5, 91, 8, 7, 99, 6]


# Algorytm roju

In [4]:
# 🎯 Funkcja dekodująca permutację do rozwiązania bin packing (heurystyka First Fit)
# Dla zadanej permutacji elementów pakuje je kolejno do pojemników tak,
# by zmieściły się bez przekroczenia pojemności BIN_CAPACITY.
def first_fit(permutation):
    bins = []
    for index in permutation:
        item = ITEM_WEIGHTS[index]
        placed = False
        for bin in bins:
            if sum(bin) + item <= BIN_CAPACITY:
                bin.append(item)
                placed = True
                break
        if not placed:
            bins.append([item])
    return bins


# 🐝 Klasa cząstki (particle) używana w algorytmie PSO
class Particle:
    def __init__(self, num_items):
        # Losowa permutacja elementów — to aktualna propozycja rozwiązania
        self.position = list(np.random.permutation(num_items))
        self.velocity = []  # lista ruchów (zamian pozycji w permutacji)
        self.best_position = self.position.copy()  # najlepsza znana permutacja dla tej cząstki
        self.best_cost = self.evaluate(self.position)  # liczba użytych pojemników dla tej permutacji

    # Funkcja celu: liczba pojemników potrzebnych dla danej permutacji
    def evaluate(self, position):
        bins = first_fit(position)
        return len(bins)

    # Aktualizacja "prędkości" (czyli zestawu zamian indeksów, które przybliżają nas do najlepszego rozwiązania)
    def update_velocity(self, global_best_position, w=0.5, c1=1.0, c2=1.0):
        swaps = []

        for i in range(NUM_ITEMS):
            # Zamiana w kierunku własnego najlepszego rozwiązania (eksploatacja)
            if random.random() < c1 and self.position[i] != self.best_position[i]:
                j = self.position.index(self.best_position[i])
                swaps.append((i, j))

            # Zamiana w kierunku najlepszego globalnego rozwiązania (eksploracja)
            if random.random() < c2 and self.position[i] != global_best_position[i]:
                j = self.position.index(global_best_position[i])
                swaps.append((i, j))

        random.shuffle(swaps)
        self.velocity = swaps[:20]  # ograniczamy liczbę zmian w jednej iteracji (siła ruchu)

    # Zastosowanie velocity do pozycji cząstki (czyli faktyczne wykonanie zamian)
    def apply_velocity(self):
        for i, j in self.velocity:
            self.position[i], self.position[j] = self.position[j], self.position[i]

    # Sprawdzenie czy nowe położenie jest lepsze — jeśli tak, aktualizujemy pamięć cząstki
    def update_personal_best(self):
        cost = self.evaluate(self.position)
        if cost < self.best_cost:
            self.best_cost = cost
            self.best_position = self.position.copy()


# 🐦 Główna funkcja algorytmu PSO dla problemu bin packing
def bin_packing_pso(num_particles=80, max_iter=200):
    start_time = time.time()

    # Inicjalizacja populacji cząstek (każda z losową permutacją)
    swarm = [Particle(NUM_ITEMS) for _ in range(num_particles)]

    # Wyznaczamy najlepszą początkową cząstkę w całym roju
    global_best = min(swarm, key=lambda p: p.best_cost)
    global_best_position = global_best.best_position.copy()
    global_best_cost = global_best.best_cost

    # Główna pętla optymalizacji
    for iteration in range(max_iter):
        for particle in swarm:
            particle.update_velocity(global_best_position)  # wyznacz ruch
            particle.apply_velocity()                      # wykonaj ruch
            particle.update_personal_best()                # aktualizuj najlepsze lokalne

        # Po aktualizacji wszystkich cząstek — sprawdź, czy któraś poprawiła globalne minimum
        best_candidate = min(swarm, key=lambda p: p.best_cost)
        if best_candidate.best_cost < global_best_cost:
            global_best_cost = best_candidate.best_cost
            global_best_position = best_candidate.best_position.copy()

        print(f"Iteracja {iteration+1}: najlepsza liczba pojemników = {global_best_cost}")

    end_time = time.time()
    final_bins = first_fit(global_best_position)

    # 📋 Zwrot wyniku jako słownik
    result = {
        "number_of_bins": len(final_bins),
        "bin_contents": final_bins,
        "execution_time_seconds": round(end_time - start_time, 2)
    }
    return result


In [5]:
# ▶️ Uruchomienie algorytmu
result = bin_packing_pso()

# 🖨️ Prezentacja wyniku
print("\n📦 Ostateczny wynik algorytmu PSO (Bin Packing):")
print(f"🔢 Liczba użytych pojemników: {result['number_of_bins']}")
print(f"⏱️ Czas wykonania: {result['execution_time_seconds']} s\n")
for i, bin in enumerate(result['bin_contents']):
    print(f"🗃️ Pojemnik {i+1}: {bin} (suma: {sum(bin)})")

Iteracja 1: najlepsza liczba pojemników = 273
Iteracja 2: najlepsza liczba pojemników = 273
Iteracja 3: najlepsza liczba pojemników = 273
Iteracja 4: najlepsza liczba pojemników = 273
Iteracja 5: najlepsza liczba pojemników = 272
Iteracja 6: najlepsza liczba pojemników = 272
Iteracja 7: najlepsza liczba pojemników = 272
Iteracja 8: najlepsza liczba pojemników = 272
Iteracja 9: najlepsza liczba pojemników = 272
Iteracja 10: najlepsza liczba pojemników = 272
Iteracja 11: najlepsza liczba pojemników = 272
Iteracja 12: najlepsza liczba pojemników = 272
Iteracja 13: najlepsza liczba pojemników = 272
Iteracja 14: najlepsza liczba pojemników = 272
Iteracja 15: najlepsza liczba pojemników = 271
Iteracja 16: najlepsza liczba pojemników = 271
Iteracja 17: najlepsza liczba pojemników = 271
Iteracja 18: najlepsza liczba pojemników = 271
Iteracja 19: najlepsza liczba pojemników = 271
Iteracja 20: najlepsza liczba pojemników = 271
Iteracja 21: najlepsza liczba pojemników = 271
Iteracja 22: najlepsza